#### address 디렉토리 하위에 templates 디렉토리 추가
#### address/templates/index.html
```address/templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Title</title>
</head>
<body>
<h2>시작 페이지</h2>
<a href="admin/">관리자</a><br>
<a href="address/">주소록</a><br>
</body>
</html>
```

#### address/views.py
```address/views.py
# Create your views here.

from django.shortcuts import redirect, render
    # redirect: 방향 전환: 작업완료 >> 새로운 작업 (글쓰기 완료 후 목록으로)
    # render: 요청하는 데이터를 화면 생성해 표시

from address.models import Address

def home(request):
        # request: 요청사항을 처리하는 내장객체
    items = Address.objects.order_by("name")
            # Address: 테이블, objects 모든 레코드들, 'name' 필드명. 이름으로 정렬
    return render(request, 'address/list.html', {'items': items, 'address_count': len(items)})
        # return: 화면으로 이동
        # address/list.html 템플릿(이동할 페이지)을 렌더링함, 템플릿 내 변수에 값 할당(데이터 전달)
        # 프로젝트/templates/address/list.html


def write(request):  # 내용을 등록
    return render(request, "address/write.html")

def insert(request):
    addr = Address(name=request.POST['name'], tel=request.POST['tel'], email=request.POST['email'], address=request.POST['address'])
        # ↑ 데이터를 만들어라. 필드명=값
        # request.POST['변수명'] : html에 <input type='text' name='변수명'> 과 같은 변수명
        # POST 서버에 보낼 때 / GET 데이터를 읽을 때
    addr.save()
    return redirect("/address/")
        # 방향전환해서 목록으로 이동해라

def detail(request):
    addr=Address.objects.get(idx=request.GET['idx'])
        # 번호가 idx인 자료를 가져와라. get방식
    return render(request, 'address/detail.html', {'addr': addr})

def update(request):
    addr = Address(idx=request.POST['idx'], name=request.POST['name'], tel=request.POST['tel'], email=request.POST['email'], address=request.POST['address'] )
        # 수정, 삽입과 비슷하나 차이점은 idx인덱스가 들어감
    addr.save()
    return redirect("/address")

def delete(request):
    Address.objects.get(idx=request.POST['idx']).delete()
        # 작업이 끝나면 지워라
    return redirect("/address")

### CRUD: 응용프로그램 기본 처리명령
# Create    레코드생성
# Read      읽기
# Update    수정
# Delete    삭제
```

#### 구조    
    urls.py         views.py                templates

    주소 요청       데이터 준비, 처리       화면

#### address/templates 디렉토리 하위에 address 디렉토리 추가
#### address/templates/address/list.html
```address/templates/address/list.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Title</title>
</head>
<body>
    <h2>주소록</h2>
    <a href="/">Home</a>
    <a href="write">등록</a>
    <table border="l">
        <tr>
            <th>번호</th>
            <th>이름</th>
            <th>전화</th>
            <th>이메일</th>
            <th>주소</th>
        </tr>
        {%for row in items %}
        <tr>
            <td>{{row.idx}}</td>
            <td><a href="detail?idx={{row.idx}}">{{row.name}}</a></td>
            <td>{{row.tel}}</td>
            <td>{{row.email}}</td>
            <td>{{row.address}}</td>
        </tr>
        {% endfor %}
    </table>

</body>
</html>
```

#### 웹 출력 GET/POST: 서버에 request 요청하는 메서드 
ex) 홈페이지에 로그인 등의 작업을 처리할 때 사용자가 폼을 작성 후 로그인 요청

- GET방식(가져온다): 클라이언트의 요청이 URL뒤에 추가되어 요청됨
    - POST보다 전송속도가 빠름
    - ex)  www.xxx.com?id=aaa&bbb=111 과 같이 작성됨
    - URL 형태로 출력되므로, 특정 페이지를 타인이 접속 가능, URL길이가 한정되어 있으므로 한계

- POST방식(수행한다): URL이 아닌 BODY에 데이터를 넣어서 숨겨진 상태로 전송
    - 보통 html에서 form형태를 이용해 submit을 사용
    - submit을 누르면 view를 거치도록 되어 있음
    - 사이버공격으로부터 방어하기 위해 form태그 끝에 {%csrf_token %}을 작성해주어야 함
    


#### address/templates/address/write.html

```address/templates/address/write.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Title</title>
  <!-- Cross Site Request Forgery 사이트 간 요청 위조 -->
</head>
<body>
<h2>주소록</h2>
<form method="post" action="insert">
  <!-- 전송  데이터받을 주소 -->
  {% csrf_token %}  <!-- 안 쓰면 에러남-->
  이 름 : <input name="name"><br>
  <!-- 변수명 -->
  전 화 : <input name="tel"><br>
  메 일 : <input name="email"><br>
  주 소 : <input name="address"><br>
  <input type="submit" value="확인">
  <!-- 제출 >> 'insert'로 감 -->
</form>
</body>
</html>
```

#### address/templates/address/detail.html
```address/templates/address/detail.html
<!--<!DOCTYPE html>-->
<!--<html lang="en">-->
<!--<head>-->
<!--    <meta charset="UTF-8">-->
<!--    <title>Title</title>-->
<!--</head>-->
<!--<body>-->

<!--</body>-->
<!--</html>-->

<!DOCTYPE html>
<html>
<head>
<script>
<!-- 자바스크립트 함수, 사용자브라우저에서 실행됨. 내장되어 있음-->
function update(){
    document.form1.action="update"
<!--  현재페이지객체 name form1 action주소   -->
    document.form1.submit();
}
function del(){
    document.form1.action="delete"
    document.form1.submit();
}
</script>
</head>
<body>
    <h2>주소록 편집</h2>
    <form method="post" name="form1">
        {% csrf_token %}
        <table border="1">
            <tr>
                <td>이름</td>
                <td><input name="name" value="{{addr.name}}"></td>
                            <!-- 변수명 -->
            </tr>
            <tr>
                <td>전화</td>
                <td><input name="tel" value="{{addr.tel}}"></td>
            </tr>
            <tr>
                <td>이메일</td>
                <td><input name="email" value="{{addr.email}}"></td>
            </tr>
            <tr>
                <td>주소</td>
                <td><input name="address" value="{{addr.address}}"></td>
            </tr>
            <tr>
                <td colspan="2" align="center">
                    <input type="hidden" name="idx" value="{{addr.idx}}">
                            <!-- hidden 정보숨김필드, 변수명, 번호 -->
                    <input type="button" value="수정" onclick="update()">
                            <!-- onclick: 버튼 클릭하면 위의 update() 자바스크립트 함수 호출해라 -->
                    <input type="button" value="삭제" onclick="del()">
                </td>
            </tr>
        </table>
    </form>
</body>
</html>
```


URL: http://127.0.0.1:8000/address/detail?idx=1

    url?key=value&key=value
        ?뒤: 쿼리스트링 

    함수에서 가져옴
```
def detail(request):
    addr = Address.objects.get(idx=request.GET['idx'])
```